# Magnetic Field Diffusion (Rm << 1)

**Validation case**: 1D magnetic field diffusion in the resistive limit

## Learning Objectives

After completing this notebook, you will understand:

1. The magnetic induction equation and the role of resistivity
2. The magnetic Reynolds number and what Rm << 1 means physically
3. How a Gaussian magnetic field profile spreads due to diffusion
4. The analytic solution for 1D magnetic diffusion
5. How to validate numerical MHD solvers against exact solutions

---
## 1. Physics Background

### The Magnetic Induction Equation

The evolution of magnetic field in a conducting plasma is governed by the **induction equation**:

$$\frac{\partial \mathbf{B}}{\partial t} = \nabla \times (\mathbf{v} \times \mathbf{B}) + \eta \nabla^2 \mathbf{B}$$

where:
- $\mathbf{v}$ is the plasma velocity
- $\eta = 1/(\mu_0 \sigma)$ is the magnetic diffusivity (related to resistivity)
- $\sigma$ is the electrical conductivity

The two terms represent competing physics:

| Term | Physics | Effect |
|------|---------|--------|
| $\nabla \times (\mathbf{v} \times \mathbf{B})$ | Advection | Field frozen to plasma, moves with flow |
| $\eta \nabla^2 \mathbf{B}$ | Diffusion | Field slips through plasma, spreads out |

### The Magnetic Reynolds Number

The **magnetic Reynolds number** measures the relative importance of advection vs diffusion:

$$R_m = \frac{v L}{\eta} = \frac{\text{advection rate}}{\text{diffusion rate}}$$

where $v$ is a characteristic velocity and $L$ is a characteristic length scale.

| Regime | Condition | Physics |
|--------|-----------|--------|
| **Ideal MHD** | $R_m \gg 1$ | Advection dominates, field frozen to plasma |
| **Resistive MHD** | $R_m \ll 1$ | Diffusion dominates, field slips through plasma |

In this notebook, we study the **resistive limit** ($R_m \ll 1$) where $\mathbf{v} = 0$ and the induction equation reduces to:

$$\frac{\partial \mathbf{B}}{\partial t} = \eta \nabla^2 \mathbf{B}$$

This is the **heat equation** (or diffusion equation) for the magnetic field!

### Analytic Solution: Spreading Gaussian

For a 1D Gaussian initial condition:

$$B_z(z, t=0) = B_{\text{peak}} \exp\left(-\frac{z^2}{2\sigma_0^2}\right)$$

The exact solution is a **spreading Gaussian**:

$$B_z(z, t) = B_{\text{peak}} \sqrt{\frac{\sigma_0^2}{\sigma_0^2 + 2\eta t}} \exp\left(-\frac{z^2}{2(\sigma_0^2 + 2\eta t)}\right)$$

Key features:
- **Width grows**: $\sigma(t) = \sqrt{\sigma_0^2 + 2\eta t}$
- **Peak decreases**: amplitude $\propto 1/\sigma(t)$ to conserve total flux
- **Characteristic timescale**: $\tau_{\text{diff}} = \sigma_0^2 / (2\eta)$

The total magnetic flux $\Phi = \int B_z \, dz$ is conserved (it just spreads out).

### Physical Intuition

Magnetic diffusion is analogous to heat conduction:

- A localized "hot spot" of magnetic field spreads into surrounding regions
- The field diffuses from high-B to low-B regions
- Higher resistivity ($\eta$) means faster diffusion
- The process is irreversible (entropy increases)

In fusion plasmas, magnetic diffusion is usually slow ($R_m \sim 10^6$ in tokamaks), but it becomes important:
- During magnetic reconnection events
- In resistive instabilities (tearing modes)
- Near plasma boundaries where temperature (and thus conductivity) is lower

---
## 2. Configuration Setup

In [ ]:
# Standard imports
import jax
import jax.numpy as jnp
import jax.lax as lax
import numpy as np
import matplotlib.pyplot as plt
from IPython.display import HTML

# jax-frc imports
from jax_frc.configurations import MagneticDiffusionConfiguration
from jax_frc.solvers.explicit import EulerSolver

# Notebook utilities
import sys
sys.path.insert(0, '.')
from _shared import plot_comparison, plot_error, compute_metrics, metrics_summary, animate_overlay

# Plotting style
plt.rcParams['figure.dpi'] = 100
plt.rcParams['font.size'] = 11

In [ ]:
# === PHYSICS PARAMETERS ===

B_peak = 1.0       # Peak magnetic field [T]
sigma = 0.3        # Initial Gaussian width [m]
eta = 1e-2         # Magnetic diffusivity [m²/s]

# === GRID PARAMETERS ===

nr = 8             # Minimal radial resolution (1D problem in z)
nz = 128           # Axial resolution
z_extent = 2.0     # Domain: z ∈ [-z_extent, z_extent]

# === TIME PARAMETERS ===

# Diffusion timescale
tau_diff = sigma**2 / (2 * eta)
print(f"Diffusion timescale: τ_diff = {tau_diff:.4f} s")

t_end = 0.1 * tau_diff    # Run for 0.1 diffusion times
dt = 1e-4 * tau_diff      # Small timestep for stability

print(f"Simulation time: t_end = {t_end:.4e} s")
print(f"Timestep: dt = {dt:.4e} s")
print(f"Expected steps: ~{int(t_end/dt)}")

In [ ]:
# Create configuration
config = MagneticDiffusionConfiguration(
    B_peak=B_peak,
    sigma=sigma,
    eta=eta,
    nr=nr,
    nz=nz,
    z_extent=z_extent,
)

# Build simulation components
geometry = config.build_geometry()
initial_state = config.build_initial_state(geometry)
model = config.build_model()
solver = EulerSolver()

print(f"Grid: {geometry.nr} × {geometry.nz} (r × z)")
print(f"Domain: z ∈ [{geometry.z_min:.1f}, {geometry.z_max:.1f}]")
print(f"Resolution: Δz = {geometry.dz:.4f}")
print(f"\nMagnetic Reynolds number: Rm = {config.magnetic_reynolds_number():.2e} (should be << 1)")

### Visualize Initial Condition

In [ ]:
# Extract 1D profile at mid-radius
r_mid = geometry.nr // 2
z = np.array(geometry.z_grid[r_mid, :])

# Initial B_z profile
Bz_init = np.array(initial_state.B[r_mid, :, 2])

# Plot
fig, ax = plt.subplots(figsize=(10, 5))

ax.plot(z, Bz_init, 'b-', linewidth=2, label='Initial $B_z$')
ax.axhline(B_peak * np.exp(-0.5), color='gray', linestyle='--', alpha=0.5, 
           label=f'Half-max (σ = {sigma})')
ax.axvline(-sigma, color='gray', linestyle=':', alpha=0.5)
ax.axvline(sigma, color='gray', linestyle=':', alpha=0.5)

ax.set_xlabel('z [m]', fontsize=12)
ax.set_ylabel('$B_z$ [T]', fontsize=12)
ax.set_title('Initial Condition: Gaussian $B_z$ Profile', fontsize=14)
ax.legend(fontsize=11)
ax.grid(True, alpha=0.3)
ax.set_xlim(-z_extent, z_extent)

plt.tight_layout()
plt.show()

print(f"Peak B_z = {np.max(Bz_init):.3f} T at z = 0")
print(f"Initial width σ₀ = {sigma} m")

---
## 3. Run Simulation

We evolve the Gaussian profile forward in time. The magnetic field should spread out according to the analytic solution.

In [ ]:
# Time stepping setup
n_steps = int(t_end / dt)
save_interval = max(1, n_steps // 50)  # ~50 snapshots

print(f"Running {n_steps} steps with dt = {dt:.2e}")
print(f"Saving every {save_interval} steps")

In [ ]:
# Run simulation using lax.scan for efficiency
@jax.jit
def scan_step(state, _):
    new_state = solver.step(state, dt, model, geometry)
    return new_state, new_state.B[r_mid, :, 2]

# Run simulation
print("Running simulation...")
final_state, Bz_history = lax.scan(scan_step, initial_state, None, length=n_steps)

# Convert to numpy and subsample for plotting
Bz_history = np.array(Bz_history)[::save_interval]
times = np.arange(0, n_steps, save_interval) * dt

# Add final state if not included
Bz_final = np.array(final_state.B[r_mid, :, 2])
if len(times) == 0 or times[-1] < t_end - dt:
    Bz_history = np.vstack([Bz_history, Bz_final])
    times = np.append(times, final_state.time)

print(f"Simulation complete: t = {final_state.time:.4e} s")
print(f"Saved {len(times)} snapshots")

---
## 4. Compare with Analytic Solution

In [ ]:
# Compute analytic solution at final time
Bz_analytic = np.array(config.analytic_solution(jnp.array(z), final_state.time))

# Expected width at final time
sigma_final = np.sqrt(sigma**2 + 2 * eta * final_state.time)
print(f"Initial width: σ₀ = {sigma:.4f} m")
print(f"Final width:   σ(t) = {sigma_final:.4f} m")
print(f"Width increase: {(sigma_final/sigma - 1)*100:.1f}%")

In [ ]:
# Overlay comparison plot
fig, ax = plot_comparison(
    z, Bz_final, Bz_analytic,
    xlabel='z [m]', ylabel='$B_z$ [T]',
    title=f'Magnetic Diffusion at t = {final_state.time:.4e} s',
    initial=Bz_init
)
plt.show()

In [ ]:
# Error distribution
fig, axes = plot_error(
    z, Bz_final, Bz_analytic,
    xlabel='z [m]',
    title='Error Analysis'
)
plt.show()

In [ ]:
# Compute validation metrics
metrics = compute_metrics(Bz_final, Bz_analytic)

# Display with thresholds
thresholds = {
    'l2_error': 0.05,      # 5% relative L2 error
    'max_rel_error': 0.10,  # 10% max relative error
}

print("Validation Metrics:")
print("=" * 40)
for name, value in metrics.items():
    threshold = thresholds.get(name, None)
    if threshold:
        status = '✓ PASS' if value <= threshold else '✗ FAIL'
        print(f"{name:20s}: {value:.4e}  (threshold: {threshold:.2e}) {status}")
    else:
        print(f"{name:20s}: {value:.4e}")

# Overall result
overall_pass = all(metrics[k] <= thresholds[k] for k in thresholds)
print("=" * 40)
print(f"Overall: {'✓ PASS' if overall_pass else '✗ FAIL'}")

---
## 5. Time Evolution Animation

In [ ]:
# Define analytic solution function for animation
def analytic_fn(z_arr, t):
    return np.array(config.analytic_solution(jnp.array(z_arr), t))

# Create animation
anim = animate_overlay(
    z, list(Bz_history), analytic_fn, times,
    xlabel='z [m]', ylabel='$B_z$ [T]',
    title='Magnetic Diffusion',
    initial=Bz_init,
    interval=100
)

HTML(anim.to_jshtml())

---
## 6. Interactive Exploration

In [ ]:
import ipywidgets as widgets
from IPython.display import display, clear_output

def explore_time(time_idx):
    """Show solution at a specific snapshot."""
    fig, axes = plt.subplots(1, 2, figsize=(14, 5))
    
    t = times[time_idx]
    Bz_num = Bz_history[time_idx]
    Bz_ana = analytic_fn(z, t)
    
    # Solution comparison
    axes[0].plot(z, Bz_init, 'gray', linestyle=':', alpha=0.7, label='Initial')
    axes[0].plot(z, Bz_num, 'b-', linewidth=2, label='Numerical')
    axes[0].plot(z, Bz_ana, 'r--', linewidth=2, label='Analytic')
    axes[0].set_xlabel('z [m]', fontsize=12)
    axes[0].set_ylabel('$B_z$ [T]', fontsize=12)
    axes[0].set_title(f'Solution at t = {t:.4e} s', fontsize=12)
    axes[0].legend(fontsize=10)
    axes[0].grid(True, alpha=0.3)
    axes[0].set_xlim(-z_extent, z_extent)
    axes[0].set_ylim(0, B_peak * 1.1)
    
    # Error
    error = np.abs(Bz_num - Bz_ana)
    axes[1].plot(z, error, 'r-', linewidth=2)
    axes[1].set_xlabel('z [m]', fontsize=12)
    axes[1].set_ylabel('Absolute Error [T]', fontsize=12)
    axes[1].set_title('Error Distribution', fontsize=12)
    axes[1].grid(True, alpha=0.3)
    axes[1].set_xlim(-z_extent, z_extent)
    
    # Add width indicator
    sigma_t = np.sqrt(sigma**2 + 2 * eta * t)
    axes[0].axvline(-sigma_t, color='green', linestyle='--', alpha=0.5)
    axes[0].axvline(sigma_t, color='green', linestyle='--', alpha=0.5, 
                    label=f'σ(t) = {sigma_t:.3f}')
    axes[0].legend(fontsize=10)
    
    plt.tight_layout()
    plt.show()
    
    # Print metrics
    m = compute_metrics(Bz_num, Bz_ana)
    print(f"L2 error: {m['l2_error']:.4e}  |  Max error: {m['linf_error']:.4e}")

# Time slider
time_slider = widgets.IntSlider(
    value=0, min=0, max=len(times)-1, step=1,
    description='Snapshot:',
    continuous_update=False
)

ui = widgets.VBox([
    widgets.HTML('<h4>Time Evolution Explorer</h4>'),
    widgets.HTML('<p>Drag the slider to see the solution at different times. Watch the Gaussian spread!</p>'),
    time_slider,
])

interactive_output = widgets.interactive_output(explore_time, {'time_idx': time_slider})

display(ui, interactive_output)

---
## Summary

This notebook demonstrated:

1. **Physics**: In the resistive limit ($R_m \ll 1$), the magnetic induction equation reduces to a diffusion equation. Magnetic field spreads through the plasma like heat through a conductor.

2. **Analytic solution**: A Gaussian initial condition evolves as a spreading Gaussian with:
   - Width: $\sigma(t) = \sqrt{\sigma_0^2 + 2\eta t}$
   - Peak: decreases as $1/\sigma(t)$ to conserve total flux

3. **Validation**: The numerical solution matches the analytic solution within the specified error tolerance, confirming correct implementation of resistive diffusion.

4. **Characteristic timescale**: $\tau_{\text{diff}} = \sigma_0^2 / (2\eta)$ sets the rate of spreading.

### Physical Significance

Magnetic diffusion is important in:
- **Magnetic reconnection**: Field lines can break and reconnect when diffusion is significant
- **Resistive instabilities**: Tearing modes, resistive wall modes
- **Edge physics**: Lower temperature means higher resistivity and faster diffusion

### Try Next

- Increase `eta` to see faster diffusion
- Increase `nz` for better resolution of the Gaussian peak
- Compare with the **frozen flux** notebook to see the opposite limit ($R_m \gg 1$)